# ICO Data

## Load raw data

In [4]:
import pandas as pd
tokendata = pd.read_csv("data/tokendata.csv").drop("Unnamed: 0", axis=1)
tokendata.loc[:, "name"] = tokendata["name"].map(str.lower) 
icotracker = pd.read_csv("data/icotracker.csv").drop("Unnamed: 0", axis=1)
icotracker.loc[:, "name"] = icotracker["name"].map(str.lower)
icostats = pd.read_csv("data/icostats.csv").drop("Unnamed: 0", axis=1)
icostats.loc[:, "name"] = icostats["name"].map(str.lower)
coinmcap = pd.read_csv("data/coinmarketcap_historical.csv").drop("Unnamed: 0", axis=1)
whitepapers = pd.read_csv("data/whitepapers_raw.csv").drop("Unnamed: 0", axis=1)
whitepapers.loc[:, "name"] = whitepapers["name"].map(str.lower)

In [5]:
valid_tokendata = tokendata[["name", "symbol", "sale_price", "curr_price"]].dropna().copy()

In [6]:
valid_tokendata.head() # BUG: names and tickers do not match. Gotta review the code

,name,symbol,sale_price,curr_price
102,propy,PRO,1.093,2.388
103,maecenas,ART,0.519,1.246
104,qtum,QTUM,0.307,38.397
105,waves,WAVES,0.188,8.779
106,ethlend,LEND,0.016,0.204


In [7]:
#df = pd.merge(whitepapers, valid_tokendata, left_on="name", right_on="name", how="left")
#df.dropna().head()

In [8]:
df = pd.merge(whitepapers, icostats[["name", "ico_price"]], left_on="name", right_on="name", how="left")
df.dropna()

,text,symbol,name,ico_price
104,b'OmiseGO\n\nDecentralized Exchange and Paymen...,OMG,omisego,0.000124
105,"b""V0.5\n\nThe\xc2\xa0CoinDash\xc2\xa0Team\n\nC...",CDT,coindash,0.000014
108,b'\x0c\xc2\xa0\n\n\xc2\xa0\n1.0\xc2\xa0Abstrac...,WGR,wagerr,0.000026
188,b'1\n\n\x0cContents\n\n2\n\n\x0c33\n\n\x0c44\n...,CVC,civic,0.000039
365,"b"" \n\n \n\nWHITEPAPER \n\nWe believe empoweri...",POWR,power ledger,0.000077
510,b' \n\n \n\n \n\n \n\nTierion Network \n\nA\xc...,TNT,tierion,0.000017
551,b'<HTML>\n<HEAD>\n<TITLE>Not Found</TITLE>\n</...,SNT,santiment,0.000104


In [9]:
# Coinmcap has the ticker included in the name,
# remove it to match the name column from other DataFrames
coinmcap.loc[:, "name"] = coinmcap["name"].map(lambda x: x.lower().split(" ")[-1])

## Approximate returns

In [10]:
# Approximate returns for each ICO
# Get the first and last date from historical price data for each ico.
first_price = coinmcap.groupby("name").apply(lambda x: x.sort_values("date", ascending=False).iloc[-1])[["price"]]
first_price.columns = ["ico_price_aprox"]
last_price = coinmcap.groupby("name").apply(lambda x: x.sort_values("date", ascending=True).iloc[-1])[["price"]]
max_price = coinmcap.groupby("name")[["price"]].max()
first_price.columns = ["max_price"]
market_cap = coinmcap.groupby("name").apply(lambda x: x.sort_values("date", ascending=True).iloc[-1])[["market_cap"]]

In [11]:
merge1 = pd.merge(whitepapers, max_price, left_on="name", right_index=True, how="left")
merge2 = pd.merge(merge1, market_cap, left_on="name", right_index=True, how="left")
merge3 = pd.merge(merge2, first_price, left_on="name", right_index=True, how="left")

In [12]:
merge3.dropna()

,text,symbol,name,price,market_cap,max_price
18,b'A DECENTRALIZED ESPORTS BETTING SYSTEM\n\nGi...,GIM,gimli,0.070379,5.630287e+06,0.020800
75,b' \n \n \n\nWhite Paper \n\n(Some details mi...,CS,credits,1.100000,3.420000e+03,0.000109
96,b'Opus - Decentralized music distribution usin...,OPT,opus,0.084597,1.185042e+07,0.047187
104,b'OmiseGO\n\nDecentralized Exchange and Paymen...,OMG,omisego,20.290001,2.070642e+09,0.439359
105,"b""V0.5\n\nThe\xc2\xa0CoinDash\xc2\xa0Team\n\nC...",CDT,coindash,0.258687,1.450328e+08,0.171627
108,b'\x0c\xc2\xa0\n\n\xc2\xa0\n1.0\xc2\xa0Abstrac...,WGR,wagerr,0.657847,1.203860e+08,0.092859
111,b' \n\nDECENTRALIZED BLOCKCHAIN LODGING ECOSYS...,EPY,emphy,0.359808,2.747819e+06,0.218479
121,"b""\x0c \n\nFirst\xe2\x80\x8b \xe2\x80\x8bInter...",PRIX,privatix,14.400000,1.702713e+07,1.840000
124,b' \n \n \n\nInternxt \n\nWelcome To A New In...,INXT,internxt,26.820000,1.688424e+07,2.890000
125,b'+\n\nWhite Paper\n\nJuly 2017\n\nSocial is a...,SOCIAL,nexus,12.220000,6.704486e+08,0.001628


In [13]:
aprox_returns = last_price / first_price

## Create pdf_paths

In [14]:
import os
import numpy as np
    
def get_paths():
    base_path = "data/all_wp/"
    subs = ["upcoming", "past", "current"]
    files = []
    for d in subs:
        files += [os.path.join(base_path, d, f) for f in os.listdir(os.path.join(base_path, d))]
    return files


In [15]:
def add_whitepaper_path(df):
    pdf_paths = get_paths()
    df["wp_path"] = np.nan
    for path in pdf_paths:
        ticker, name = path.split("/")[-1].split(".")[0].split("_")
        cond_1 = df["name"].values == name.lower()
        cond_2 = df["symbol"].values == ticker
        df.loc[cond_1 & cond_2, "wp_path"] = path
    return df

In [16]:
data = add_whitepaper_path(merge3)

In [17]:
#save data
data_final = pd.merge(data, icotracker[["name", "scam"]], left_on="name", right_on="name", how="left")
data_final.to_csv("ICO_data.csv")